In [1]:
# Jupyter Notebook for rapid prototyping and debugging individual modules (e.g., testing PDF ingestion, RAG hint generation)
import pandas as pd
import requests
import json


In [2]:
!pwd

/Users/pedram/Projects/GitMyself/AITutorApp/docs/notebooks


In [3]:
# load questions.csv
pd.read_csv('../../data/questions.csv')

,id,question,question_type,options,correct_answer,skill
0,1,We want to distinguish between three different...,multiple_choice,"[""scipy, numpy, matplotlib, pandas, sklearn"",""...",3,[Data Loading]
1,2,"load the dataset \n url = ""https://raw.githubu...",multiple_choice,"[""dataset = read_csv(url, names=names)"",""datas...",1,[Data Loading]
2,3,Which one is not true about the data?,multiple_choice,"[""150 samples, with 4 attributes (same units, ...",4,[Exploratory Data Analysis (EDA)]
3,4,Which of the following methods are not suitabl...,multiple_choice,"[""Logistic Regression (LR), Linear Discriminan...",4,[Supervised Learning]
4,5,Which of the following is not required in IRIS...,multiple_choice,"[""Set-up test harness"",""Handling missing data""...",2,[Data Preprocessing]
5,6,fill the blank to import a library to do binar...,"""fill_in_the_blank""",NaN,"""LogisticRegression""","""[Supervised Learning]"""


In [11]:
BASE_URL = "http://127.0.0.1:8000"
USER_ID = "notebook_user_session"

# 1. Create the user first
print(f"--- Creating user: {USER_ID} ---")
create_user_response = requests.post(f"{BASE_URL}/users/", json={"user_id": USER_ID})
print(f"Status Code: {create_user_response.status_code}")
print(create_user_response.json())
print("-" * 20)

--- Creating user: notebook_user_session ---
Status Code: 200
{'user_id': 'notebook_user_session', 'created_at': '2025-10-05T01:58:19.225633', 'preferences': {'hint_style_preference': 'adaptive', 'intervention_preference': 'manual'}, 'feedback_scores': {}, 'skill_mastery': [], 'interaction_history': []}
--------------------


In [12]:
# stage 1
print("--- Requesting a hint ---")
payload = {
   "user_id": USER_ID,
   "question_number": 5,
   "user_answer": "I think the answer is 1 we do not need to set-up test harness"
    }
headers = {
"Content-Type": "application/json"
   }
response = requests.post(f"{BASE_URL}/hints/", headers=headers, data=json.dumps(payload))
print(f"Status Code: {response.status_code}")
print(response.json())

--- Requesting a hint ---
Status Code: 200
{'question_number': 5, 'hint': "The question is asking about what's *not* needed when working with the Iris dataset. Think about the fundamental steps involved in using a dataset for", 'user_id': 'notebook_user_session', 'hint_style': 'Conceptual', 'pre_hint_mastery': 0.2}


In [13]:
# stage 2
print("\nSubmitting answer...")
payload = {
   "user_id": USER_ID,
   "question_number": 1,
   "user_answer": "2",
   "time_taken_ms": 15000
   }
headers = {
   "Content-Type": "application/json"
}

try:
   response = requests.post(f"{BASE_URL}/answer/", headers=headers, data=json.dumps(payload))
   response.raise_for_status()
   print(f"Answer submission response: {response.status_code}")
   print(response.json())
except requests.exceptions.RequestException as e:
   print(f"Error submitting answer: {e}")
   if e.response:
        print(f"Response content: {e.response.text}")



Submitting answer...
Answer submission response: 200
{'correct': False, 'correct_answer': '3', 'skill': '[Data Loading]', 'intervention_needed': True, 'current_mastery': 0.17575757575757575}


In [14]:
# Stage 3: Checking the Consolidated User Profile
# 1. Ensure the user exists (or create them)
requests.post(f"{BASE_URL}/users/", json={"user_id": USER_ID})
# 2. Check the consolidated user profile
print(f"--- Checking profile for user: {USER_ID} ---")
profile_url = f"{BASE_URL}/users/{USER_ID}/profile"
profile_response = requests.get(profile_url)
print(f"Status Code for /profile: {profile_response.status_code}")
print("Response for /profile:", json.dumps(profile_response.json(), indent=2))

--- Checking profile for user: notebook_user_session ---
Status Code for /profile: 200
Response for /profile: {
  "user_id": "notebook_user_session",
  "created_at": "2025-10-05T01:58:19.225633",
  "preferences": {
    "hint_style_preference": "adaptive",
    "intervention_preference": "manual"
  },
  "feedback_scores": {},
  "skill_mastery": [
    {
      "skill_id": "[Data Loading]",
      "mastery_level": 0.17575757575757575,
      "consecutive_errors": 1
    }
  ],
  "interaction_history": [
    {
      "timestamp": "2025-10-05T01:58:23.414304",
      "question_id": 1,
      "skill": "[Data Loading]",
      "user_answer": "2",
      "is_correct": false,
      "time_taken_ms": 15000,
      "hint_shown": false,
      "hint_style_used": null,
      "user_feedback_rating": null,
      "bkt_change": null
    }
  ]
}


In [15]:
# Stage 4: Answer Submission and Verifying BKT Update

# 1. Ensure the user exists
requests.post(f"{BASE_URL}/users/", json={"user_id": USER_ID})

# 2. Submit an answer to a question
print("--- Submitting an answer ---")
answer_payload = {
   "user_id": USER_ID,
   "question_number": 1,
   "user_answer": "4" # Assuming '4' is an incorrect answer for question 1
    }
answer_url = f"{BASE_URL}/answer/"
answer_response = requests.post(answer_url, json=answer_payload)

print(f"Status Code for /answer: {answer_response.status_code}")
print("Response for /answer:", json.dumps(answer_response.json(), indent=2))
print("-" * 20)

# 3. Check the user's profile to see the updated BKT mastery
print("--- Checking profile for BKT update ---")
profile_url = f"{BASE_URL}/users/{USER_ID}/profile"
profile_response = requests.get(profile_url) 
print(f"Status Code for /profile: {profile_response.status_code}")
print("Response for /profile:", json.dumps(profile_response.json(), indent=2))

--- Submitting an answer ---
Status Code for /answer: 200
Response for /answer: {
  "correct": false,
  "correct_answer": "3",
  "skill": "[Data Loading]",
  "intervention_needed": true,
  "current_mastery": 0.1720680393912265
}
--------------------
--- Checking profile for BKT update ---
Status Code for /profile: 200
Response for /profile: {
  "user_id": "notebook_user_session",
  "created_at": "2025-10-05T01:58:19.225633",
  "preferences": {
    "hint_style_preference": "adaptive",
    "intervention_preference": "manual"
  },
  "feedback_scores": {},
  "skill_mastery": [
    {
      "skill_id": "[Data Loading]",
      "mastery_level": 0.1720680393912265,
      "consecutive_errors": 2
    }
  ],
  "interaction_history": [
    {
      "timestamp": "2025-10-05T01:58:37.961615",
      "question_id": 1,
      "skill": "[Data Loading]",
      "user_answer": "4",
      "is_correct": false,
      "time_taken_ms": null,
      "hint_shown": false,
      "hint_style_used": null,
      "user_fee

In [16]:
# Stage 5:
user_id = USER_ID
base_url = BASE_URL

preferences_payload = {
"hint_style_preference": "Worked Example",
 "intervention_preference": "proactive"
    }
preferences_url = f"{base_url}/users/{user_id}/preferences"
preferences_response = requests.put(preferences_url, json=preferences_payload)
 
print(f"Status Code for PUT /preferences: {preferences_response.status_code}")
print("Response for PUT /preferences:", json.dumps(preferences_response.json(), indent=2))
print("-" * 20)

# 2. Request a hint to see if the new preference is used
hint_payload = {
 "user_id": user_id,
  "question_number": 2
}
hint_url = f"{base_url}/hints/"
hint_response = requests.post(hint_url, json=hint_payload)

print(f"Status Code for /hints: {hint_response.status_code}")
hint_data = hint_response.json()
print("Response for /hints:", json.dumps(hint_data, indent=2))
print("-" * 20)

# 3. Submit feedback on the hint that was just received
print("--- Submitting answer with feedback ---")
answer_with_feedback_payload = {
   "user_id": USER_ID,
   "question_number": 2,
   "user_answer": "Some answer after the hint",
   "hint_shown": True,
   "feedback_rating": 3, # This is where the 1-5 rating now goes
   "pre_hint_mastery": hint_data.get("pre_hint_mastery"),
   "hint_style_used": hint_data.get("hint_style"),
   "hint_text": hint_data.get("hint")
   }
answer_url = f"{BASE_URL}/answer/"
answer_response = requests.post(answer_url, json=answer_with_feedback_payload)
print(f"Status Code for /answer: {answer_response.status_code}")
print("Response for /answer:", json.dumps(answer_response.json(), indent=2))

Status Code for PUT /preferences: 200
Response for PUT /preferences: {
  "hint_style_preference": "Worked Example",
  "intervention_preference": "proactive"
}
--------------------
Status Code for /hints: 200
Response for /hints: {
  "question_number": 2,
  "hint": "Okay, let's break down how to approach loading a dataset and assigning column names. Here's a similar example to guide you:\n\n**",
  "user_id": "notebook_user_session",
  "hint_style": "Worked Example",
  "pre_hint_mastery": 0.1720680393912265
}
--------------------
--- Submitting answer with feedback ---
Status Code for /answer: 200
Response for /answer: {
  "correct": false,
  "correct_answer": "1",
  "skill": "[Data Loading]",
  "intervention_needed": true,
  "current_mastery": 0.1715226727794904
}


In [17]:
# 2. Seed feedback to train the personalization model
print("--- Seeding feedback to train the model ---")
# --- Seed Loop 1: Give 'Analogy' a high rating ---
# a) Set preference to get an Analogy hint
requests.put(f"{BASE_URL}/users/{USER_ID}/preferences", json={"hint_style_preference": "Analogy"})
# b) Request the hint
hint_resp_1 = requests.post(f"{BASE_URL}/hints/", json={"user_id": USER_ID, "question_number": 3})
hint_data_1 = hint_resp_1.json()
# c) Answer with a high rating, including the required hint data
requests.post(f"{BASE_URL}/answer/", json={
     "user_id": USER_ID, "question_number": 3, "user_answer": "2",
    "hint_shown": True, "feedback_rating": 1,
     "pre_hint_mastery": hint_data_1.get("pre_hint_mastery"),
      "hint_style_used": hint_data_1.get("hint_style"),
      "hint_text": hint_data_1.get("hint")
})
print("Seeded 'Analogy' with a low rating.")

# --- Seed Loop 2: Give 'Socratic Question' a low rating ---
# a) Set preference to get a Socratic hint
requests.put(f"{BASE_URL}/users/{USER_ID}/preferences", json={"hint_style_preference": "Socratic Question"})
# b) Request the hint
hint_resp_2 = requests.post(f"{BASE_URL}/hints/", json={"user_id": USER_ID, "question_number": 4})
hint_data_2 = hint_resp_2.json()
# c) Answer with a low rating, including the required hint data
requests.post(f"{BASE_URL}/answer/", json={
    "user_id": USER_ID, "question_number": 4, "user_answer": "2",
    "hint_shown": True, "feedback_rating": 5,
    "pre_hint_mastery": hint_data_2.get("pre_hint_mastery"),
    "hint_style_used": hint_data_2.get("hint_style"),
    "hint_text": hint_data_2.get("hint")
   })
print("Seeded 'Socratic Question' with a high rating.")
print("-" * 20)

# 3. Test Adaptive Selection (Exploitation)
print("--- Testing Adaptive Hint Selection (Exploitation) ---")
# Set preference to "Automatic" to enable the adaptive logic
requests.put(f"{BASE_URL}/users/{USER_ID}/preferences", json={"hint_style_preference": "Automatic"})

# Request a hint. With the seeded data, the epsilon-greedy algorithm should exploit the best-known style ("Analogy").
hint_payload = {"user_id": USER_ID, "question_number": 5}
hint_response = requests.post(f"{BASE_URL}/hints/", json=hint_payload)

print(f"Status Code for adaptive /hints: {hint_response.status_code}")
print("Response for adaptive /hints:", json.dumps(hint_response.json(), indent=2))

--- Seeding feedback to train the model ---
Seeded 'Analogy' with a low rating.
Seeded 'Socratic Question' with a high rating.
--------------------
--- Testing Adaptive Hint Selection (Exploitation) ---
Status Code for adaptive /hints: 200
Response for adaptive /hints: {
  "question_number": 5,
  "hint": "Think about the different steps involved in working with a dataset. Consider what you need to load the data, explore it, and build a model. Which",
  "user_id": "notebook_user_session",
  "hint_style": "Automatic",
  "pre_hint_mastery": 0.2
}


In [18]:
print("\n--- Testing Post-Hint BKT Performance Tracking ---")
profile_url = f"{BASE_URL}/users/{USER_ID}/profile"
profile_response = requests.get(profile_url) 
print(f"Status Code for /profile: {profile_response.status_code}")
print("Response for /profile (skill mastery key):", json.dumps(profile_response.json().get("skill_mastery"), indent=2))
print("-" * 20)
# First, get a hint for a new question to set the pre-hint mastery
hint_payload = {"user_id": USER_ID, "question_number": 6} # Using a new question
hint_response = requests.post(f"{BASE_URL}/hints/", json=hint_payload)
hint_data = hint_response.json()
print(f"Hint given for Question 6 was style: '{hint_data.get('hint_style')}'")

# Now, submit a correct answer, flagging that a hint was shown
# and including the required context from the hint response.
answer_payload = {
   "user_id": USER_ID,
   "question_number": 6,
   "user_answer": "LogisticRegression",  # Assuming 'LogisticRegression' is correct for question 6
   "time_taken_ms": 1000,
   "hint_shown": True,
   # --- ADDED REQUIRED FIELDS ---
    "pre_hint_mastery": hint_data.get("pre_hint_mastery"),
   "hint_style_used": hint_data.get("hint_style"),
   "hint_text": hint_data.get("hint")
   }
answer_response = requests.post(f"{BASE_URL}/answer/", json=answer_payload)
print(f"Status Code for /answer: {answer_response.status_code}")
print("Response for /answer:", json.dumps(answer_response.json(), indent=2))
print("\nVERIFICATION: Check the user's profile again. The 'skill_mastery' for this question's skill should have increased.")
print("-" * 20)
profile_url = f"{BASE_URL}/users/{USER_ID}/profile"
profile_response = requests.get(profile_url) 
print(f"Status Code for /profile: {profile_response.status_code}")
print("Response for /profile (skill mastery key):", json.dumps(profile_response.json().get("skill_mastery"), indent=2))


--- Testing Post-Hint BKT Performance Tracking ---
Status Code for /profile: 200
Response for /profile (skill mastery key): [
  {
    "skill_id": "[Data Loading]",
    "mastery_level": 0.1715226727794904,
    "consecutive_errors": 3
  },
  {
    "skill_id": "[Exploratory Data Analysis (EDA)]",
    "mastery_level": 0.17575757575757575,
    "consecutive_errors": 1
  },
  {
    "skill_id": "[Supervised Learning]",
    "mastery_level": 0.17575757575757575,
    "consecutive_errors": 1
  }
]
--------------------
Hint given for Question 6 was style: 'Automatic'
Status Code for /answer: 200
Response for /answer: {
  "correct": true,
  "correct_answer": "LogisticRegression",
  "skill": "[Supervised Learning]",
  "intervention_needed": false,
  "current_mastery": 0.5662288930581614
}

VERIFICATION: Check the user's profile again. The 'skill_mastery' for this question's skill should have increased.
--------------------
Status Code for /profile: 200
Response for /profile (skill mastery key): [
  

In [19]:
#stage 6: Expanded User Model & History

USER_ID2 = "notebook_user2_session"

# 1. Create the user first
print(f"--- Creating user: {USER_ID2} ---")
create_user_response = requests.post(f"{BASE_URL}/users/", json={"user_id": USER_ID2})
print(f"Status Code: {create_user_response.status_code}")
print(create_user_response.json())
print("-" * 20)

def print_response(name, response):
    """Helper function to print formatted API responses."""
    print(f"--- {name} ---")
    print(f"Status Code: {response.status_code}")
    try:
            print(json.dumps(response.json(), indent=2))
    except json.JSONDecodeError:
            print(response.text)
    print("-" * (len(name) + 8))
    print()
def run_stage6_test():
    """Runs the test scenario for the expanded user model"""
    print(f"--- Running Test Scenario for Stage 6 with User ID: {USER_ID2} ---")
    # 1. Ensure user exists
    requests.post(f"{BASE_URL}/users/", json={"user_id": USER_ID2})
    print(f"User '{USER_ID2}' is ready.")

    # 2. Answer a multiple_choice question incorrectly
    mc_payload = {
    "user_id": USER_ID2,
    "question_number": 1,
    "user_answer": "1",  # Correct is 3
    }
    response = requests.post(f"{BASE_URL}/answer/", json=mc_payload)
    print_response("2. Answer Multiple Choice (Incorrect)", response)
    
    # 3. Answer a fill_in_the_blank question incorrectly
    fitb_payload = {
            "user_id": USER_ID2,
            "question_number": 6,
            "user_answer": "Berlin",  # Correct is "LogisticRegression"
        }
    response = requests.post(f"{BASE_URL}/answer/", json=fitb_payload)
    print_response("3. Answer Fill-in-the-Blank (Incorrect)", response)
    
    # 4. Request a hint for the fill-in-the-blank question
    # The RAG agent should now have the context of the two previous incorrect answers.
    print(">>> NOTE: Check the server logs to see the 'user_history' block sent to the LLM. <<<")
    hint_payload = {
            "user_id": USER_ID2,
            "question_number": 6,
   }
    response = requests.post(f"{BASE_URL}/hints/", json=hint_payload)
    print_response("4. Request Hint (with history)", response) 
    # 5. Retrieve the user's profile to verify the interaction history
    response = requests.get(f"{BASE_URL}/users/{USER_ID2}/profile")
    print_response("5. Get User Profile (with history)", response)
    print("--- Test Scenario for Stage 6 Complete ---")
    print("\nVERIFICATION:")
    print("1. Check server logs for 'user_history' in the prompt.")
    print("2. Check the profile output above. The interaction_history should contain records for both the incorrect answer and the hint request.")
    print("3. The first record should have 'hint_shown': false, and the second should have 'hint_shown': true.")

# To run this test, ensure the FastAPI server is running, then execute this function.
run_stage6_test()



--- Creating user: notebook_user2_session ---
Status Code: 200
{'user_id': 'notebook_user2_session', 'created_at': '2025-10-05T01:59:32.464881', 'preferences': {'hint_style_preference': 'adaptive', 'intervention_preference': 'manual'}, 'feedback_scores': {}, 'skill_mastery': [], 'interaction_history': []}
--------------------
--- Running Test Scenario for Stage 6 with User ID: notebook_user2_session ---
User 'notebook_user2_session' is ready.
--- 2. Answer Multiple Choice (Incorrect) ---
Status Code: 200
{
  "correct": false,
  "correct_answer": "3",
  "skill": "[Data Loading]",
  "intervention_needed": true,
  "current_mastery": 0.17575757575757575
}
---------------------------------------------

--- 3. Answer Fill-in-the-Blank (Incorrect) ---
Status Code: 200
{
  "correct": false,
  "correct_answer": "LogisticRegression",
  "skill": "[Supervised Learning]",
  "intervention_needed": true,
  "current_mastery": 0.17575757575757575
}
-----------------------------------------------

>>> N

In [20]:
 #  Intervention is 'manual' (Should NOT intervene) ---
print("---  Intervention preference is 'manual' (Should NOT intervene) ---")
# Set intervention preference to manual
requests.put(f"{BASE_URL}/users/{USER_ID}/preferences", json={"intervention_preference": "manual"})
 
payload1 = {
        "user_id": USER_ID,
        "question_number": 1,
        "time_spent_ms": 90000  # A very long time
  }
response1 = requests.post(f"{BASE_URL}/intervention-check", json=payload1)
print(f"Status Code: {response1.status_code}") 
print("Response:", response1.json())
print("-" * 20)
   
# --- Setup for time-based tests: Answer correctly to raise mastery ---
print("--- Setup: Answering correctly to isolate time-based trigger ---")
# This raises the user's mastery above the initial intervention threshold
requests.post(f"{BASE_URL}/answer/", json={"user_id": USER_ID, "question_number": 2, "user_answer": "1"})
print("Setup complete.")
print("-" * 20)
# --- Test 2: Intervention is 'proactive', but time is too short (Should NOT intervene) ---
print("--- Test 2: Intervention is 'proactive', time is short (Should NOT intervene) ---")
# Set intervention preference to proactive
requests.put(f"{BASE_URL}/users/{USER_ID}/preferences", json={"intervention_preference": "proactive"})
payload2 = {
        "user_id": USER_ID,
        "question_number": 2, # Use the same question as the setup
        "time_spent_ms": 1000  # A very short time
    }
response2 = requests.post(f"{BASE_URL}/intervention-check", json=payload2)
print(f"Status Code: {response2.status_code}")
print("Response:", response2.json())
print("-" * 20)
# --- Test 3: Intervention is 'proactive' and time is long (Should intervene) ---
print("--- Test 3: Intervention is 'proactive', time is long (Should intervene) ---")
payload3 = {
        "user_id": USER_ID,
        "question_number": 2, # Use the same question
        "time_spent_ms": 90000  # A very long time
   }
response3 = requests.post(f"{BASE_URL}/intervention-check", json=payload3)
print(f"Status Code: {response3.status_code}")
print("Response:", response3.json())
print("-" * 20)


---  Intervention preference is 'manual' (Should NOT intervene) ---
Status Code: 200
Response: {'intervention_needed': False}
--------------------
--- Setup: Answering correctly to isolate time-based trigger ---
Setup complete.
--------------------
--- Test 2: Intervention is 'proactive', time is short (Should NOT intervene) ---
Status Code: 200
Response: {'intervention_needed': False}
--------------------
--- Test 3: Intervention is 'proactive', time is long (Should intervene) ---
Status Code: 200
Response: {'intervention_needed': True}
--------------------


In [24]:
#clean up
print(f"--- Deleting user: {USER_ID} ---")
response = requests.delete(f"{BASE_URL}/users/{USER_ID}")
print(f"Status Code: {response.status_code}")
print(response.json())

--- Deleting user: notebook_user_session ---
Status Code: 404
{'detail': 'User not found'}
